# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

**1.1 Импортируем необходимые библиотеки.**

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

**1.2 Пользовательские функции.**

In [2]:
# Напишем функцию для первичной проверки данных

def check_data(data_df):
    print ('\033[1m' + 'Изучим исходные данные'+ '\033[0m')
    print(data_df.info())
    print(data_df.shape)
        
    missed_cells = data_df.isnull().sum().sum()/(data_df.shape[0]*(data_df.shape[1]-1))
    missed_rows = sum(data_df.isnull().sum(axis = 1)>0)/data_df.shape[0]
    print ('\033[1m' + '\nПроверка пропусков'+ '\033[0m')
    print ('Количество пропусков: {:.0f}'.format(data_df.isnull().sum().sum()))
    print ('Доля пропусков: {:.1%}'.format(missed_cells)+ '\033[0m')
    print ('Доля строк содержащих пропуски: {:.1%}'.format(missed_rows))

    ## Проверим дубликаты
    
    print ('\033[1m' + '\nПроверка на дубликаты'+ '\033[0m')
    print('Количество полных дубликатов: ', data_df.duplicated().sum())
        
    ## Посмотрим на сами данные
    
    print ('\033[1m' + '\nПервые пять строк датасета'+ '\033[0m')
    display(data_df.head())
    
    print('\033[1m' + '\nОписание количественных данных:'+ '\033[0m')
    display(data_df.describe().T)

**1.3 Загрузим данные.**

In [3]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [4]:
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


**1.4 Первичный осмотр данных.**

In [5]:
# ОПИСАТЕЛЬНЫЙ АНАЛИЗ

check_data(df)

Изучим исходные данные
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None
(3214, 5)

Проверка пропусков
Количество пропусков: 0
Доля пропусков: 0.0%
Доля строк содержащих пропуски: 0.0%

Проверка на дубликаты
Количество полных дубликатов:  0

Первые пять строк датасета


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0



Описание количественных данных:


,count,mean,std,min,25%,50%,75%,max
calls,3214.0,63.038892,33.236368,0.0,40.0000,62.000,82.0000,244.00
minutes,3214.0,438.208787,234.569872,0.0,274.5750,430.600,571.9275,1632.06
messages,3214.0,38.281269,36.148326,0.0,9.0000,30.000,57.0000,224.00
mb_used,3214.0,17207.673836,7570.968246,0.0,12491.9025,16943.235,21424.7000,49745.73
is_ultra,3214.0,0.306472,0.461100,0.0,0.0000,0.000,1.0000,1.00


**Вывод:**
- в таблице 5 столбцов и 3214 строк;
- пропусков - нет, дубликатов - нет;
- предобработка данных не требуется 

## Разбейте данные на выборки

In [6]:
# Разделим исходные данные на обучающую (60%), валидационную (20%) и тестовую (20%) выборки.

df_train, df_test = train_test_split(df, test_size=0.40, random_state=12345) 

df_valid, df_test = train_test_split(df_test, test_size=0.50, random_state=12345)

In [7]:
# проверим

print(df_train.shape[0])
print(df_test.shape[0])
print(df_valid.shape[0])

1928
643
643


**Вывод:**

Спрятанной тестовой выборки нет, поэтому разделим данные на 3 части (3:1:1):
- для начала разделим на обучающую и тестовую (60% и 40%) выборки 
- затем тестовую выборку разделили пополам - на тестовую и валидационную (20% и 20%).

## Исследуйте модели

- В нашей задаче целевой признак - is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).
- Признаки - столбцы с данными о поведении одного пользователя за месяц (количество и длительность звонков, количество смс и потраченного интернет-трафика. 

In [8]:
# для обучающей выборки

train_features = df_train.drop(['is_ultra'], axis=1)
train_target = df_train['is_ultra']

# для тестовой выборки

test_features = df_test.drop(['is_ultra'], axis=1)
test_target = df_test['is_ultra']

# для валидационной выборки

valid_features = df_valid.drop(['is_ultra'], axis=1)
valid_target = df_valid['is_ultra']

**3.1 Исследуем модель - _Решающее дерево._**

In [9]:
best_result_dtc = 0
best_depth_dtc = 0

for depth_dtc in range(1,11):
    
    # инициализируем модель DecisionTreeClassifier с параметром random_state=12345 и max_depth=depth
    
    model_dtc = DecisionTreeClassifier(random_state=12345, max_depth=depth_dtc) 
    
    # обучим модель на тренировочной выборке
    
    model_dtc.fit(train_features, train_target) 
    
    # получим предсказания модели на валидационной выборке 
    
    predictions_valid_dtc = model_dtc.predict(valid_features) 
    
    # посчитаем качество модели на валидационной выборке
    
    result_dtc = accuracy_score(valid_target, predictions_valid_dtc)
    
    
    if result_dtc > best_result_dtc:
        best_result_dtc = result_dtc
        best_depth_dtc = depth_dtc
        
print('Accuracy наилучшей модели на валидационной выборке:', best_result_dtc, 'глубина дерева -', best_depth_dtc)


Accuracy наилучшей модели на валидационной выборке: 0.7853810264385692 глубина дерева - 3


**3.2 Исследуем модель - _Случайный лес_.**

In [10]:
best_result_rfr = 0
best_est_rfr = 0
best_depth_rfr = 0

for est_rfr in range(1, 51):
    for depth_rfr in range (1, 11):
        
        # инициализируем модель RandomForestClassifier 
        # с параметрами random_state=12345, n_estimators=est и max_depth=depth
        
        model_rfr = RandomForestClassifier(random_state=12345, n_estimators=est_rfr, max_depth=depth_rfr) 
        
        # обучим модель на тренировочной выборке
        
        model_rfr.fit(train_features, train_target) 
        
        # получим предсказания модели на валидационной выборке
        
        predictions_valid_rfr = model_rfr.predict(valid_features)
        
        # посчитаем качество модели на валидационной выборке
        
        result_rfr = accuracy_score(valid_target, predictions_valid_rfr)
        
    
        if result_rfr > best_result_rfr:
            best_result_rfr = result_rfr
            best_depth_rfr = depth_rfr
            best_est_rfr = est_rfr
            
            
print('Accuracy наилучшей модели на валидационной выборке:', best_result_rfr, 
      'глубина дерева -', best_depth_rfr, 'количество деревьев - ', best_est_rfr
     )

Accuracy наилучшей модели на валидационной выборке: 0.8087091757387247 глубина дерева - 8 количество деревьев -  40


In [15]:
print('Accuracy наилучшей модели на валидационной выборке:', best_result_rfr,\
      'глубина дерева -', best_depth_rfr, 'количество деревьев - ', best_est_rfr)

Accuracy наилучшей модели на валидационной выборке: 0.8087091757387247 глубина дерева - 8 количество деревьев -  40


**3.3 Исследуем модель - _Логистическая регрессия_.**

In [12]:
# инициализируем модель LogisticRegression с параметрами random_state=12345, solver='lbfgs'и max_iter=1000

model_lr = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)

# обучим модель на тренировочной выборке

model_lr.fit(train_features, train_target) 

# получим предсказания модели на валидационной выборке

predictions_valid_lr = model_lr.predict(valid_features)

# посчитаем качество модели на валидационной выборке

result_lr = accuracy_score(valid_target, predictions_valid_lr)

print('Accuracy наилучшей модели на валидационной выборке:', result_lr)

Accuracy наилучшей модели на валидационной выборке: 0.7107309486780715


**Вывод:**
- Самая точная модель - **Случайный лес** - 80,8% правильных ответов;
- Самый низкий результат у модели - **Логистическая регрессия** - 71% правильных ответов.

## Проверьте модель на тестовой выборке

In [13]:
# проверим модель Случайный лес, т.к. она показала самый точный результат

test_predictions = model_rfr.predict(test_features)
test_accuracy = accuracy_score(test_target, test_predictions)

print('Accuracy наилучшей модели на тестовой выборке:', test_accuracy)


Accuracy наилучшей модели на тестовой выборке: 0.8009331259720062


**Вывод:**
- Точность модели **Случайный лес** на тестовой выборке = 80,1% и практически равна точности модели на валидационной выборке.

## (бонус) Проверьте модели на адекватность

In [14]:
# проверим на модели DummyClassifier со стратегией most_frequent

dummy_clf = DummyClassifier(random_state=12345, strategy='most_frequent')
dummy_clf.fit(train_features, train_target)
predictions_valid_dclf = dummy_clf.predict(valid_features)
result_clf = accuracy_score(valid_target, predictions_valid_dclf)
print('Accuracy тривиальной модели:', result_clf)

Accuracy тривиальной модели: 0.7060653188180405


**Вывод:**

- Точность тривиальной модели - 70%, она ниже, чем у выбранной нами модели - **Случайный лес**.
- Модель **Случайный лес** - адекватна.

## Общий вывод.
**Изучена общая информация по таблице:**
- В таблице 5 столбцов и 3214 строк;
- Пропусков - нет, дубликатов - нет;
- Предобработка данных не потребовалась; 

**Разбили данные на выборки:**
- На обучающую (60%), валидационную (20%) и тестовую (20%) выборки;

**Исследовали модели:**
- Исследовали три модели - Решающее дерево, Случайный лес и Логистическую регрессию;
- Самая точная модель - **Случайный лес** - 80,8% правильных ответов;
- Самый низкий результат у модели - **Логистическая регрессия** - 71% правильных ответов.

**Проверили модель на тестовой выборке:**
- Точность модели **Случайный лес** на тестовой выборке = 80,1% и практически равна точности модели на валидационной выборке.

**Проверили модель на адекватность:**
- Точность тривиальной модели - 70%, она ниже, чем у выбранной нами модели - **Случайный лес**.
- Модель **Случайный лес** - адекватна.




## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [ ] Весь код исполняется без ошибок
- [ ] Ячейки с кодом расположены в порядке исполнения
- [ ] Выполнено задание 1: данные загружены и изучены
- [ ] Выполнено задание 2: данные разбиты на три выборки
- [ ] Выполнено задание 3: проведено исследование моделей
    - [ ] Рассмотрено больше одной модели
    - [ ] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [ ] Написаны выводы по результатам исследования
- [ ] Выполнено задание 3: Проведено тестирование
- [ ] Удалось достичь accuracy не меньше 0.75
